In [ ]:
"""

Credits:

https://www.kaggle.com/code/emiz6413/llama-3-8b-38-faster-inference
https://www.kaggle.com/code/kishanvavdara/inference-llama-3-8b
https://www.kaggle.com/code/emiz6413/inference-gemma-2-9b-4-bit-qlora

LB: 0.945
"""

In [ ]:
!pip install transformers peft accelerate bitsandbytes -U --no-index --find-links /kaggle/input/lmsys-wheel-files

In [ ]:
import time
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor

import torch
import sklearn
import numpy as np
import pandas as pd
from transformers import (
    Gemma2ForSequenceClassification, GemmaTokenizerFast, 
    AutoTokenizer, LlamaForSequenceClassification, BitsAndBytesConfig
)
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
from peft import PeftModel, get_peft_model, LoraConfig, TaskType

torch.backends.cuda.enable_mem_efficient_sdp(True)
torch.backends.cuda.enable_flash_sdp(True)

In [ ]:
@dataclass
class Config:
    gemma_dir = '/kaggle/input/gemma-2/transformers/gemma-2-9b-it-4bit/1/gemma-2-9b-it-4bit'
    gemma_lora_dir = '/kaggle/input/73zap2gx/checkpoint-5748'
    llama_model_name = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'
    llama_weights_path = '/kaggle/input/lmsys-model/model'
    max_length = 2048
    batch_size = 4
    tta = False
    spread_max_length = False

cfg = Config()

In [ ]:
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

def process_text(text: str) -> str:
    return " ".join(eval(text, {"null": ""}))

test.loc[:, 'prompt'] = test['prompt'].apply(process_text)
test.loc[:, 'response_a'] = test['response_a'].apply(process_text)
test.loc[:, 'response_b'] = test['response_b'].apply(process_text)

In [ ]:
def tokenize(tokenizer, prompt, response_a, response_b, max_length=cfg.max_length, spread_max_length=cfg.spread_max_length):
    if isinstance(tokenizer, GemmaTokenizerFast):
        prompt = ["<prompt>: " + p for p in prompt]
        response_a = ["\n\n<response_a>: " + r_a for r_a in response_a]
        response_b = ["\n\n<response_b>: " + r_b for r_b in response_b]
    else:
        prompt = ["User prompt: " + p for p in prompt]
        response_a = ["\n\nModel A :\n" + r_a for r_a in response_a]
        response_b = ["\n\n--------\n\nModel B:\n" + r_b for r_b in response_b]
    
    if spread_max_length:
        prompt = tokenizer(prompt, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_a = tokenizer(response_a, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_b = tokenizer(response_b, max_length=max_length//3, truncation=True, padding=False).input_ids
        input_ids = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        attention_mask = [[1]* len(i) for i in input_ids]
    else:
        text = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        tokenized = tokenizer(text, max_length=max_length, truncation=True, padding=False)
        input_ids = tokenized.input_ids
        attention_mask = tokenized.attention_mask
    return input_ids, attention_mask

In [ ]:
# Gemma Tokenizer
gemma_tokenizer = GemmaTokenizerFast.from_pretrained(cfg.gemma_dir)
gemma_tokenizer.add_eos_token = True
gemma_tokenizer.padding_side = "right"

# Llama Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/lmsys-model/tokenizer')

# Prepare data for both models
gemma_data = pd.DataFrame()
gemma_data["id"] = test["id"]
gemma_data["input_ids"], gemma_data["attention_mask"] = tokenize(gemma_tokenizer, test["prompt"], test["response_a"], test["response_b"])
gemma_data["length"] = gemma_data["input_ids"].apply(len)

llama_data = pd.DataFrame()
llama_data["id"] = test["id"]
llama_data["input_ids"], llama_data["attention_mask"] = tokenize(llama_tokenizer, test["prompt"], test["response_a"], test["response_b"])
llama_data["length"] = llama_data["input_ids"].apply(len)

In [ ]:
# Load Gemma model on GPU 0
device_0 = torch.device('cuda:0')
gemma_model = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    device_map=device_0,
    use_cache=False
)
gemma_model = PeftModel.from_pretrained(gemma_model, cfg.gemma_lora_dir)

In [ ]:
# Load Llama model on GPU 1
device_1 = torch.device('cuda:1')
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16,
    bnb_8bit_use_double_quant=False
)
llama_base_model = LlamaForSequenceClassification.from_pretrained(
    cfg.llama_model_name,
    num_labels=3,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    device_map='cuda:1')
llama_base_model.config.pad_token_id = llama_tokenizer.pad_token_id

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.10,
    bias='none',
    inference_mode=True,
    task_type=TaskType.SEQ_CLS,
    target_modules=['o_proj', 'v_proj']
)
llama_model = get_peft_model(llama_base_model, peft_config).to(device_1)
llama_model.load_state_dict(torch.load(cfg.llama_weights_path), strict=False)
llama_model.eval()

In [ ]:
@torch.no_grad()
@torch.cuda.amp.autocast()
def inference(df, model, tokenizer, device, batch_size=cfg.batch_size, max_length=cfg.max_length):
    a_win, b_win, tie = [], [], []
    
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        tmp = df.iloc[start_idx:end_idx]
        input_ids = tmp["input_ids"].to_list()
        attention_mask = tmp["attention_mask"].to_list()
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",
            pad_to_multiple_of=None,
            return_tensors="pt",
        )
        outputs = model(**inputs.to(device))
        proba = outputs.logits.softmax(-1).cpu()
        
        a_win.extend(proba[:, 0].tolist())
        b_win.extend(proba[:, 1].tolist())
        tie.extend(proba[:, 2].tolist())
    
    df["winner_model_a"] = a_win
    df["winner_model_b"] = b_win
    df["winner_tie"] = tie
    
    return df

In [ ]:
st = time.time()

# Sort data by input length
gemma_data = gemma_data.sort_values("length", ascending=False)
llama_data = llama_data.sort_values("length", ascending=False)

with ThreadPoolExecutor(max_workers=2) as executor:
    results = executor.map(inference, 
                           (gemma_data, llama_data), 
                           (gemma_model, llama_model),
                           (gemma_tokenizer, llama_tokenizer),
                           (device_0, device_1))

gemma_result_df, llama_result_df = list(results)

# Combine results (simple average)
combined_result_df = gemma_result_df.copy()
combined_result_df["winner_model_a"] = (gemma_result_df["winner_model_a"] + llama_result_df["winner_model_a"]) / 2
combined_result_df["winner_model_b"] = (gemma_result_df["winner_model_b"] + llama_result_df["winner_model_b"]) / 2
combined_result_df["winner_tie"] = (gemma_result_df["winner_tie"] + llama_result_df["winner_tie"]) / 2

print(f"Inference time: {time.time() - st:.2f} seconds")

In [ ]:
submission_df = combined_result_df[["id", 'winner_model_a', 'winner_model_b', 'winner_tie']]
submission_df.to_csv('submission.csv', index=False)
display(submission_df.head())